[[source]](../api/alibi.explainers.html#alibi.explainers.PartialDependenceVariance)

# Partial Dependence Variance

<a id='source_1'></a>
<a id='source_2'></a>
<a id='source_3'></a>
<a id='source_4'></a>
<a id='source_5'></a>
<a id='source_6'></a>


## Overview

Partial Dependence Variance is a method proposed by [Greenwell et al. (2018)](https://arxiv.org/abs/1805.04755)[[1]](#Reference) to compute the global feature importance or the feature interaction of a pair of features, respectively. As the naming suggest, the the feature importance and the feature interactions are summarized in a single positive number given by the variance within the [Partial Dependence (PD)](https://projecteuclid.org/journals/annals-of-statistics/volume-29/issue-5/Greedy-function-approximation-A-gradient-boostingmachine/10.1214/aos/1013203451.full)[[2, 3]](#Reference) function. Because the computation relies on the PD, the method is quite intuitive and easy to comprehend.

To get a better intuition of what the proposed method tries to achieve, let us consider a simple example. Given a trained model on the Bike rental dataset, one can compute the PD function for each individual feature. The following plot contains the PD for 6 out of 11 features:

<img src="pd_variance_intro_pdp_bike.png" alt="PD plots Bike rental dataset." width="1000"/>


From the inspection of the plots, we can observe that temperature (temp), humidity (hum), wind speed (windspeed) have a strong non-linear relationship with the predicted outcome. We can observe that the model prediction increases with temperature till it reaches approx $17^{\circ}C$. Then it flattens at a high number until the weather becomes too hot(i.e. approx. $27^{\circ}C$ ), after which it starts dropping again. The humidity larger than $60\%$ seems to be a factor that inhibits the number of rentals, since we can observe a downward trend form that point onward. Finally, a similar analysis can be conducted for speed. As the wind speed increases, fewer and fewer people are riding the bike.


Quite noticeable are the plots in the second row which show a flat response. Naturally, although some heterogeneity can be hidden, one can assume that the the features in the second row have a less impact on the model prediction than the others.

Given the arguments above, one can propose a notion of quantifying the importance of a feature based on a measure of flatness of the PD function, for which the variance represents a natural and straightforward candidate. The following figure displays the global feature importance for the given example using the variance of the partial dependence function (left figure) and a model-internal method (i.e., based on impurity) (left figure):

<img src="pd_variance_intro_importance_comparison_bike.png" alt="Feature importance comparison between the PD variance and impurity-based method." width="1000">

As we can observe, the two methods agree on the to most salient features.

**Advantages**:

* the method offers a standardize procedure to quantify the feature importance for any learning algorithm.  This contrasts with some internal feature importance for some tree-based algorithms such as [Random Forest](https://link.springer.com/article/10.1023/a:1010933404324)[[5]](#References) or [Gradient Boosting](https://www.jstor.org/stable/2699986)[[6]](#References), which have their own way to define the importance of a feature.

* the method operates in the black-box regime (i.e., can be applied to any prediction model).

* the method can be adapted to quantify the strength of potential interaction effects.

**Drawbacks**:

* since the computation of the feature importance is based on the PD, the method captures only the main effect of a feature and ignores possible feature interactions. The PDP can be flat as the feature affects the predictions manly through interactions. This is related to the masked heterogeneity.

* the method can fail to detect feature interactions even though those exist (see theoretical overview).

## Usage

<a id='source_1'></a>

## Theoretical exposition

We will split the theoretical exposition in two parts, the first one covering the feature importance and the second one covering the feature interaction


### Feature importance

Following the notation from the [Partial Dependence exposition](https://docs.seldon.io/projects/alibi/en/stable/methods/PartialDependence.html#Theoretical-exposition), we said that any variable with a flat PDP is likely to be less important than those for which the PDP varies across a wider range. This notion of variable importance is based on a measure of the flatness of the PD function which can be generally stated as:

$$
i(x) = F(f_{S}(x_s)),
$$

where $F(\cdot)$ is any measure of the "flatness" for the PD of the variables $S$.

[Greenwell et al. (2018)](https://arxiv.org/abs/1805.04755)[[1]](#Reference) proposed to measure the "flatness" as the sample standard deviation for continuous features and the range statistic divided by four for the categorical features. Note that the range divided by four is an estimate of the standard deviation for a small sample size. Formally, those statistics are defined as:


\begin{equation}
 i(x) =
  \begin{cases}
   \sqrt{\frac{1}{k-1}\sum_{i=1}^{k}[f_{1}(x_{1i}) - \frac{1}{k}\sum_{j=1}^{k}f_{1}(x_{1i})]^2} \text{, if } x_{1} \text{ is continous} \\
  [\max_{i}(f_{1}(x_{1i})) - \min_{i}(f_{1}(x_{1i}))] / 4 \text{, if } {x_1} \text{ is categorical}
  \end{cases}
\end{equation}


#### Connection to t-statistic

Although the choice of computing the variance of the PD can be motivated intuitively, one can justify it more rigorously by considering a linear mode as follows:

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p + \epsilon
$$


where $\beta_i$, $i = 1, ..., p$ are the regression coefficients and $\epsilon \sim \mathcal{N}(0, \sigma^2)$.

To test the significance of a regression coefficient for a least squares problem, one can apply the t-test (TODO: cite). For that, one has to compute the t-statistic given by: 

$$
\text{t-statistic} = \frac{\hat\beta_i - \beta_{H_0}}{s.e.(\hat\beta_i)},
$$

where $\hat\beta_i$ is an estimate of $\beta_i$, $\beta_{H_0}$ is the value under the null hypothesis, and $s.e.$ is the standard error.

If we set $\beta_{H_0} = 0$, then the t-statistic is given by:

$$
\text{t-statistic} = \frac{\hat\beta_i}{s.e.(\beta_i)}.
$$

For completeness, we will provide a sketch of the derivation of the t-statistic for the least squares problem. Using the matrix notation, we can rewrite the least squares problem as follows:

$$
\hat\beta = \arg\min_{\beta \in \mathbb{R}^p} \lvert Y - X \beta \rvert_{2}^2,
$$

where $Y$ is the target variable. The least squares solution of the equation above is given by:

$$
\hat\beta = (X^TX)^{-1}X^TY.
$$

Under the assumption that the true model is given by $Y = X\beta + \epsilon$, we can infer the distribution of $\hat\beta$:

$$
\hat\beta = (X^TX)^{-1}X^T(X\beta + \epsilon) = \beta + (X^TX)^{-1}X^T\epsilon.
$$

Thus, one can conclude that $\hat\beta - \beta \sim \mathcal{N}(0, \sigma^2(X^TX)^{-1})$. Knowing that $\hat\beta - \beta$ has a multivariate normal distribution, we can look at the diagonal entrance and obtain that $\hat\beta_i - \beta_i \sim \mathcal{N}(0, \sigma^2 S_{ii})$, where $S_{ii}$ is the i-th diagonal entrance on of the matrix $(X^TX)^{-1}$. The last statement implies that:

$$
z_i = \frac{\hat\beta_i - \beta_i}{\sqrt{\sigma^2 S_{ii}}} \sim \mathcal{N}(0, 1).
$$


Let us denote by $\hat\epsilon = (I - X(X^TX)^{-1}X^T)Y$ be the residuals, and $s^2 = \frac{\epsilon^T\epsilon}{n-p}$ be an unbiased estimate of $\sigma^2$. One can show that:

$$
V = \frac{(n-p)s^2}{\sigma^2} \sim \chi_{n-p}^{2}.
$$

Given that $z_i \sim \mathcal{N}(0, 1)$ and $V \sim \chi_{n-p}^2$, we can conclude that $t_i = \frac{z_i}{\sqrt{V / (n -p)}}$ is characterized by a t-student distribution with $n-p$ degrees of freedom. With some simple algebraic manipulation, one can show that $t_i = \frac{\hat{\beta}_i - \beta_i}{s.e.(\hat{\beta}_i)}$.

$$
t_i = \frac{\frac{\hat\beta_i - \beta_i}{\sqrt{\sigma^2 S_{ii}}}}{\sqrt{\frac{(n-p)s^2}{\sigma^2} / (n-p)}} 
= \frac{\frac{\hat\beta_i - \beta_i}{\sqrt{S_{ii}}}}{\sqrt{s^2}} = \frac{\hat\beta_i - \beta_i}{\sqrt{s^2 S_{ii}}} = \frac{\hat\beta_i - \beta_i}{s.e.(\hat\beta_i)} 
$$


To see exactly the connection between the Partial Dependence Variance feature importance and the t-statistic, we consider the following example also presented in [Greenwell et al. (2018)](https://arxiv.org/abs/1805.04755)[[1]](#Reference):

$$
Y = \hat\beta_0 + \hat\beta X_1 + \hat\beta X_2
$$

where $\hat\beta_0, \hat\beta_1, \hat\beta_2$ are constants, $X_1$ and $X_2$ are both independent $\mathcal{U}[0, 1]$. Since the distributions of $X_1$ and $X_2$ are known, one can compute the exact $f_i(X_i)$. For example, $f_1(X_1) = \int_{0}^{1} \mathbb{E}[Y | X_1, X_2] p(X_2)dX_2$, where $p(X_2) = 1$. After simple calculus, one obtains:

$$
f_1(X_1) = \hat\beta_0 + \frac{\hat\beta_2}{2} + \hat\beta_1 X_1 \text{ and } f_2(X_2) = \hat\beta_0 + \frac{\hat\beta_1}{2} + \hat\beta_2 X_2.
$$

Computing the variance for each PD function above, gives us:

$$
\mathbb{V}[f_1(X_1)] = \frac{\hat\beta_{1}^{2}}{12} \text{ and } \mathbb{V}[f_2(X_2)] = \frac{\hat\beta_{2}^{2}}.{12}
$$

which implies that the standard deviation is given by $\frac{\lvert \hat\beta_1 \rvert}{\sqrt{12}}$ and $\frac{\vert \hat\beta_2 \rvert}{\sqrt{12}}$, respectively.

On the other hand, the two-tailed t-statistic is given by:

$$
t_1 = \frac{\lvert \hat\beta_1 \rvert}{\sqrt{s^2 S_{11}}} = \frac{\lvert \hat\beta_1 \rvert}{\sqrt{12 s^2}} \text{ and } t_2 = \frac{\lvert \hat\beta_2 \rvert}{\sqrt{s^2 S_{22}}} = \frac{\lvert \hat\beta_2 \rvert}{\sqrt{12 s^2}}
$$

which matches the Partial Dependence Variance up to a proportionality constant. Thus, the variance of the PD measures the significance of each regression coefficients and orders them accordingly. In other words, the most important features will correspond to the the ones with the most significant p-values.


### Feature interaction
 
[Greenwell et al. (2018)](https://arxiv.org/abs/1805.04755)[[1]](#Reference) also proposed use the PD to compute measure the feature interaction of two given features. Let $SD(X_i, X_j)$, with $i \neq j$, be the standard deviation of the joint PD values $f_{ij}(X_{ii^\prime}, X_{jj^\prime})$, for $i^\prime = 1, 2, ..., k_i$ and $j^\prime = 1, 2, ..., k_j$. The intuition proposed by the authors is that a weak interaction effect between $X_i$ and $X_j$ on the response $Y$ would suggest that importance $i(X_i, X_j)$ has little variance when either $X_i$ or $X_j$ is held constant and the other varies.


The computation of the feature interaction is straight forward. Consider $(X_i, X_j)$, $i \neq j$ be any two features. We construct the PD function $f_{ij}(X_i, X_j)$ and compute the feature importance of $X_i$ while keeping $X_j$ constant, for all values of $X_j$. We denote it by $SD(X_i | X_j)$. Following that, we take the standard deviation of the resulting importance scores across all values of $X_j$. We denote the latter quantity as $i(X_i | X_j)$. Similarly, we compute $i(X_j | X_i)$. To compute the feature interaction, one simply averages the two results. A large values will indicate a possible feature interactions.

Although the results reported by [Greenwell et al. (2018)](https://arxiv.org/abs/1805.04755)[[1]](#Reference) seem encouraging, the authors do not offer a rigorous justification for their proposal which makes the method to appear rather heuristic. In the following paragraphs, we will try to provide through a concrete examples some arguments on why the method can capture feature interactions and which are some failure cases.


Consider the following function of two variables, $f: [0, 1] \rightarrow \mathbb{R}$, $f(X_1, X_2) = X_1 + X_2 + X_1 X_2$. Due to its simplistic form, one might be tempted to say eyeball the decomposition of the function in three terms: a main effect of $X_1$ given by $f_1(X_1)$, a main effect of $X_2$ given by $f_2(X_2)$, and an interaction term between $(X_1, X_2)$ given by $f_3(X_1, X_2) = X_1 X_2$. Although this can be a valid function decomposition within an axiomatic framework, this is not the case for the Partial Dependence. This is because in the PD case the term $X_1 X_2$ does not only contain a feature interaction between $X_1$ and $X_2$, but also contains a fraction of their main effects.

To understand why $X_1 X_2$ contains also a fraction of the main effects of $X_1$ and $X_2$, we will first provide an intuitive view of what the main effect consists when using the PD functions. Informally, one can think of the main effect of a feature in the PD context as how well w.r.t. the mean squared error one can approximate $f(X_1, X_2)$ by only having access to the feature $X_1$. In our case we can analyze the three terms:

* for $f_1(X_1) = X_1$ is straight forward. Since we have access to the feature $X_1$, we can approximate the function exactly.


* for $f_2(X_2) = X_2$ is also relatively easy. Since we only have access to $X_1$ and because $f_2(X_2)$ does not depend on $X_1$, the best we can do is to approximate it with a constant. The constant that we choose is dependent of the objective we want to minimize. For MSE, the constant is given by $\mathbb{E}[f_2(X_2)]$, where the expectation is take w.r.t. the marginal of $X_2$.


* for $f_3(X_1, X_2) = X_1 X_2$ is a bit more challenging. As mentioned before, one might be tempted to say that $f_3$ describes only the feature interaction between $X_1$ and $X_2$, but given our PD approach, this is not the case. This is because in the PD context, the $f_3$ contains also a fractions of the main effects of $X_1$ and $X_2$. 

We will now elaborate more on the third bullet point. To see why this is the case, let us fix the value $X_1=0.2$  and inspect $f_3(0.2, X_2)$ by varying $X_2$ (see Figure 2(a)).

<img src="pd_variance_estimation_steps.png" alt="Conditional PD estimation steps." width="1000">

Having no access to $X_2$, the best we can predict $f_3(X_1, X_2)$ based only on $X_1 = 0.2$ w.r.t. the MSE, is to predict the average response of $f_3(0.2, X_2)$ over the marginal of $X_2$. Formally, the value is given by $\mathbb{E}_{X_2}[f_3(0.2, X_2)]$. This is given by the green line in Figure 2(b).

The residuals (i.e. what we cannot predict) are given by $f_3(0.2, X_2) - \mathbb{E}[f_3(0.2, X_2)]$, displayed in red in Figure 2(b).

The residuals/errors can be attribute to the followings:

* either to a fraction from the main effect of $X_2$.

* either to the interaction between $X_1$ and $X_2$.

* or a combination of both.


Intuitively, we would conclude that we do not have any feature interaction between $X_1$ and $X_2$ if for any other constant value of $X_1 = c$, we would obtain the same error patterns as above. In this case, the errors above would only be a result from a fraction of the main effect of $X_2$. In the following figure displays those the two scenarios (first row - without interaction and second row - with interaction).

<img src="pd_variance_error_patterns.png" alt="Conditional PD estimation error patterns." width="700">


Coming back to [Greenwell et al. (2018)](https://arxiv.org/abs/1805.04755)[[1]](#Reference) proposal to measure the feature interaction. For feature $X_1$, the first step is to compute the standard deviation of the PD along the $X_2$ axis when $X_1$ is held constant. This is equivalent of computing the root mean squared error (i.e., corresponds to the red lines above). This first step can quantify whether there is some effect we are missing just by using $X_1$, which we can attribute to either a fraction from the main effect of $X_2$ or to the interaction between $X_1$ and $X_2$ (we still do not know to which one we should attribute). We repeat the same step for every value of $X_1$. Having all the standard deviations for every value of $X_{1i}$ (i.e., $SD_{X_2}(X_2 | X_{1i})$, we compute the standard deviation along the $X_1$, $SD_{X_1}[SD_{X_2}(X_2 | X_{1})]$. This step is necessary (but not sufficient) to check whether the variance in the error is attributed to the fraction from the main effect of $X_2$ or to the feature interaction between $X_1$ and $X_2$.  Note that if the standard deviation $SD_{X_1}[SD(X_2 | X_1)] = 0$, it means that we have the same variance along the $X_2$ for every value of $X_{1i}$. This might happen for multiple reasons, but one reason can be if we encounter the same error pattern when we condition on every $X_{1i}$. If that happens then it means that there is no feature interaction and all the error is attributed to a fraction of the main effect of $X_2$.

One simple example for which the method fails to capture the feature interaction is for $f:[-1, 1] \times [-1, 1] \rightarrow \mathbb{R}$, $f(X_1, X_2) = I(X_1 X_2 > 0)$. In this case the variance is the same, but the error patterns when keeping $X_1$ or $X_2$ constants, depending on their sign. The PD and the conditional importances can be visualized in the following figure:

<img src="pd_variance_interaction_failure.png" alt="Feature interaction failure case." width="1000">


Although there is clear interaction between $X_1$ and $X_2$, the method fails to detect it because the variance along the $X_1$ and $X_2$ axis stays the same.




## Examples

[PD Variance, regression example (Friedman's regression problem)](../examples/pd_variance_regression_friedman.ipynb)

## References

<a id='References'></a>

[[1]](#source_1) Greenwell, Brandon M., Bradley C. Boehmke, and Andrew J. McCarthy. "A simple and effective model-based variable importance measure." arXiv preprint arXiv:1805.04755 (2018).

[[2]](#source_2) Friedman, Jerome H. "Greedy function approximation: a gradient boosting machine." Annals of statistics (2001): 1189-1232.

[[3]](#source_3) Molnar, Christoph. Interpretable machine learning. Lulu. com, 2020.

[[4]](#source_4) Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg.

[[5]](#source_5) Breiman, Leo. "Random forests." Machine learning 45.1 (2001): 5-32.

[[6]](#source_6) Friedman, Jerome H. "Greedy function approximation: a gradient boosting machine." Annals of statistics (2001): 1189-1232.